# 3rd_2002

## Raw Data

In [1]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_3rd_governor_election(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None
):
    """
    선거 데이터를 유연하게 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, optional
        필터링할 값 (예: '합계')

    Returns:
    --------
    pandas.DataFrame
        처리된 데이터프레임

    Example:
    --------
    # 2~4행을 헤더로, '읍면동명'이 '합계'인 행만 추출
    df = process_3rd_governor_election(
        'https://github.com/.../강원도지사선거.xlsx',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value='합계'
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 파일을 header 없이 읽기
    df_raw = pd.read_excel(file_path_or_url, header=None)

    # 엑셀 행 번호를 파이썬 인덱스로 변환 (엑셀은 1부터, 파이썬은 0부터)
    start_idx = header_rows[0] - 1
    end_idx = header_rows[1] - 1

    # 지정된 행들을 가져와서 컬럼명 생성
    header_rows_data = []
    for i in range(start_idx, end_idx + 1):
        header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = []

        # 각 행의 값이 '_'가 아닌 경우만 추가
        for row_data in header_rows_data:
            if row_data[col_idx] != '_':
                parts.append(row_data[col_idx])

        # parts가 비어있으면 '_', 아니면 '_'로 연결
        if parts:
            new_columns.append('_'.join(parts))
        else:
            new_columns.append('_')

    # 데이터프레임 재구성 (헤더 다음 행부터 데이터로 사용)
    data_start_idx = end_idx + 1
    df = df_raw.iloc[data_start_idx:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 컬럼명 샘플 출력
    print("\n생성된 컬럼명 (처음 10개):")
    for i, col in enumerate(new_columns[:10]):
        print(f"{i}: {col}")

    # 필터링 적용
    if filter_column and filter_value:
        # 필터링할 컬럼 찾기
        matching_col = None
        for col in df.columns:
            if filter_column in col:
                matching_col = col
                break

        if matching_col:
            print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")

            # 필터링 적용
            filtered_df = df[df[matching_col] == filter_value].copy()

            print(f"필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
            print(f"'{matching_col}' == '{filter_value}'인 행만 추출")

            return filtered_df
        else:
            print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")
            print("필터링 없이 전체 데이터를 반환합니다.")

    return df


# 사용 예시

# 1. 강원도지사선거 - 2~4행을 헤더로, '읍면동명'이 '합 계'인 행만
# url = "https://github.com/.../강원도지사선거.xlsx"
# summary_df = process_3rd_governor_election(
#     url,
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value='합 계'
# )

# 2. 다른 파일 - 1~3행을 헤더로, 필터링 없이
# df_all = process_3rd_governor_election(
#     'other_file.xlsx',
#     header_rows=(1, 3)
# )

# 3. 로컬 파일 - 기본값 사용
# df = process_3rd_governor_election('강원도지사선거.xlsx')

## Seoul


In [2]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_seoul = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%84%9C%EC%9A%B8.xls'

seoul_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_seoul,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 14
데이터 행 수: 3191

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이명박
5: 민주당 김민석
6: 녹색평화당 임삼진
7: 민주노동당 이문옥
8: 사회당 원용수
9: 무소속 이경희

'투표구명' 컬럼 발견: 투표구명
필터링 전: 3191행 → 필터링 후: 25행
'투표구명' == '합계'인 행만 추출


In [3]:
seoul_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이명박,민주당 김민석,녹색평화당 임삼진,민주노동당 이문옥,사회당 원용수,무소속 이경희,_,계,투 표 율 (%),유효 투표 율 (%)
0,종로구,합계,141157,72241,37592,30353,584,1928,305,689,NaN,71451,51.2,98.9
84,중구(서울),합계,110996,56999,28220,25799,410,1167,190,516,NaN,56302,51.4,98.8
155,용산구,합계,186438,89215,47878,36435,686,2154,338,823,NaN,88314,47.9,99
249,성동구,합계,259275,122030,60188,54742,965,3274,461,1184,NaN,120814,47.1,99
360,광진구,합계,288387,128061,65409,56266,904,2873,448,1121,NaN,127021,44.4,99.2
479,동대문구,합계,293982,139856,71954,59919,1034,3398,550,1592,NaN,138447,47.6,99
607,중랑구,합계,328660,141503,72168,60936,1239,3212,529,1873,NaN,139957,43.1,98.9
741,성북구,합계,344694,159058,78071,70160,1876,4328,639,2405,NaN,157479,46.1,99
899,강북구,합계,272559,119468,56515,55030,1263,3439,461,1520,NaN,118228,43.8,99
1010,도봉구,합계,267451,124314,64873,52444,1108,3227,470,1159,NaN,123281,46.5,99.2


In [4]:
seoul_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 이명박',
 '민주당 김민석',
 '녹색평화당 임삼진',
 '민주노동당 이문옥',
 '사회당 원용수',
 '무소속 이경희',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [5]:
rename_seoul = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 이명박': '득표수_1_한나라당_이명박',
    '민주당 김민석': '득표수_2_새천년민주당_김민석',
    '녹색평화당 임삼진': '득표수_3_녹색평화당_임삼진',
    '민주노동당 이문옥': '득표수_4_민주노동당_이문옥',
    '사회당 원용수': '득표수_5_사회당_원용수',
    '무소속 이경희': '득표수_6_무소속_이경희',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [6]:
seoul_3rd = seoul_3rd.rename(columns=rename_seoul).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
seoul_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_이명박,득표수_2_새천년민주당_김민석,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계
0,종로구,141157,72241,37592,30353,584,1928,305,689,71451
84,중구(서울),110996,56999,28220,25799,410,1167,190,516,56302
155,용산구,186438,89215,47878,36435,686,2154,338,823,88314
249,성동구,259275,122030,60188,54742,965,3274,461,1184,120814
360,광진구,288387,128061,65409,56266,904,2873,448,1121,127021
479,동대문구,293982,139856,71954,59919,1034,3398,550,1592,138447
607,중랑구,328660,141503,72168,60936,1239,3212,529,1873,139957
741,성북구,344694,159058,78071,70160,1876,4328,639,2405,157479
899,강북구,272559,119468,56515,55030,1263,3439,461,1520,118228
1010,도봉구,267451,124314,64873,52444,1108,3227,470,1159,123281


In [7]:
seoul_3rd = seoul_3rd.assign(
    시도='서울특별시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + seoul_3rd.columns.tolist()]

In [8]:
seoul_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이명박,득표수_2_새천년민주당_김민석,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계
0,서울특별시,종로구,141157,72241,37592,30353,584,1928,305,689,71451
84,서울특별시,중구,110996,56999,28220,25799,410,1167,190,516,56302
155,서울특별시,용산구,186438,89215,47878,36435,686,2154,338,823,88314
249,서울특별시,성동구,259275,122030,60188,54742,965,3274,461,1184,120814
360,서울특별시,광진구,288387,128061,65409,56266,904,2873,448,1121,127021


In [9]:
seoul_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 0 to 3066
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                25 non-null     object
 1   구시군               25 non-null     object
 2   선거인수              25 non-null     object
 3   투표수               25 non-null     object
 4   득표수_1_한나라당_이명박    25 non-null     object
 5   득표수_2_새천년민주당_김민석  25 non-null     object
 6   득표수_3_녹색평화당_임삼진   25 non-null     object
 7   득표수_4_민주노동당_이문옥   25 non-null     object
 8   득표수_5_사회당_원용수     25 non-null     object
 9   득표수_6_무소속_이경희     25 non-null     object
 10  득표수_계             25 non-null     object
dtypes: object(11)
memory usage: 2.9+ KB


In [10]:
seoul_3rd = seoul_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [11]:
seoul_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이명박,득표수_2_새천년민주당_김민석,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계,무효투표수,기권수
0,서울특별시,종로구,141157,72241,37592,30353,584,1928,305,689,71451,790,68916
84,서울특별시,중구,110996,56999,28220,25799,410,1167,190,516,56302,697,53997
155,서울특별시,용산구,186438,89215,47878,36435,686,2154,338,823,88314,901,97223
249,서울특별시,성동구,259275,122030,60188,54742,965,3274,461,1184,120814,1216,137245
360,서울특별시,광진구,288387,128061,65409,56266,904,2873,448,1121,127021,1040,160326


In [12]:
# 수치형 열만 합계 구하기
summary_row = seoul_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '서울특별시')

# summary_row를 맨 위에 붙이기
seoul_3rd_with_total = pd.concat([summary_row, seoul_3rd], ignore_index=True)

In [13]:
seoul_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이명박,득표수_2_새천년민주당_김민석,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계,무효투표수,기권수
0,서울특별시,합계,7665343,3510898,1819057,1496754,28034,87965,12982,34313,3479105,31793,4154445
1,서울특별시,종로구,141157,72241,37592,30353,584,1928,305,689,71451,790,68916
2,서울특별시,중구,110996,56999,28220,25799,410,1167,190,516,56302,697,53997
3,서울특별시,용산구,186438,89215,47878,36435,686,2154,338,823,88314,901,97223
4,서울특별시,성동구,259275,122030,60188,54742,965,3274,461,1184,120814,1216,137245
5,서울특별시,광진구,288387,128061,65409,56266,904,2873,448,1121,127021,1040,160326
6,서울특별시,동대문구,293982,139856,71954,59919,1034,3398,550,1592,138447,1409,154126
7,서울특별시,중랑구,328660,141503,72168,60936,1239,3212,529,1873,139957,1546,187157
8,서울특별시,성북구,344694,159058,78071,70160,1876,4328,639,2405,157479,1579,185636
9,서울특별시,강북구,272559,119468,56515,55030,1263,3439,461,1520,118228,1240,153091


In [14]:
seoul_3rd_with_total.to_csv("temp1_governor_seoul_3.csv", index=False, encoding="utf-8-sig")


## Busan


In [15]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_busan = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EB%B6%80%EC%82%B0.xls'

busan_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_busan,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1283

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 안상영
5: 민주당 한이헌
6: 민주노동당 김석준
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1283행 → 필터링 후: 16행
'투표구명' == '합계'인 행만 추출


In [16]:
busan_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 안상영,민주당 한이헌,민주노동당 김석준,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(부산),합계,45490,23688,15669,4039,3487,NaN,23195,52.1,97.9
40,서구(부산),합계,116858,47313,31336,8397,6594,NaN,46327,40.5,97.9
113,동구(부산),합계,97322,44749,29853,7808,6247,NaN,43908,46,98.1
173,영도구,합계,136484,59273,37299,12417,8545,NaN,58261,43.4,98.3
244,부산진구,합계,320729,152469,95573,28774,25497,NaN,149844,47.5,98.3
392,동래구,합계,219090,92495,59940,15482,15783,NaN,91205,42.2,98.6
481,남구(부산),합계,230494,99050,62976,17767,16906,NaN,97649,43,98.6
579,북구(부산),합계,221411,82584,49151,18109,14128,NaN,81388,37.3,98.6
667,해운대구,합계,289845,114749,69300,22401,21305,NaN,113006,39.6,98.5
779,기장군,합계,56539,32048,20143,5616,5583,NaN,31342,56.7,97.8


In [17]:
busan_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 안상영',
 '민주당 한이헌',
 '민주노동당 김석준',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [18]:
rename_busan = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 안상영': '득표수_1_한나라당_안상영',
    '민주당 한이헌': '득표수_2_새천년민주당_한이헌',
    '민주노동당 김석준': '득표수_3_민주노동당_김석준',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [19]:
busan_3rd = busan_3rd.rename(columns=rename_busan).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
busan_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_안상영,득표수_2_새천년민주당_한이헌,득표수_3_민주노동당_김석준,득표수_계
0,중구(부산),45490,23688,15669,4039,3487,23195
40,서구(부산),116858,47313,31336,8397,6594,46327
113,동구(부산),97322,44749,29853,7808,6247,43908
173,영도구,136484,59273,37299,12417,8545,58261
244,부산진구,320729,152469,95573,28774,25497,149844
392,동래구,219090,92495,59940,15482,15783,91205
481,남구(부산),230494,99050,62976,17767,16906,97649
579,북구(부산),221411,82584,49151,18109,14128,81388
667,해운대구,289845,114749,69300,22401,21305,113006
779,기장군,56539,32048,20143,5616,5583,31342


In [20]:
busan_3rd = busan_3rd.assign(
    시도='부산광역시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + busan_3rd.columns.tolist()]

In [21]:
busan_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상영,득표수_2_새천년민주당_한이헌,득표수_3_민주노동당_김석준,득표수_계
0,부산광역시,중구,45490,23688,15669,4039,3487,23195
40,부산광역시,서구,116858,47313,31336,8397,6594,46327
113,부산광역시,동구,97322,44749,29853,7808,6247,43908
173,부산광역시,영도구,136484,59273,37299,12417,8545,58261
244,부산광역시,부산진구,320729,152469,95573,28774,25497,149844


In [22]:
busan_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 1198
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                16 non-null     object
 1   구시군               16 non-null     object
 2   선거인수              16 non-null     object
 3   투표수               16 non-null     object
 4   득표수_1_한나라당_안상영    16 non-null     object
 5   득표수_2_새천년민주당_한이헌  16 non-null     object
 6   득표수_3_민주노동당_김석준   16 non-null     object
 7   득표수_계             16 non-null     object
dtypes: object(8)
memory usage: 1.7+ KB


In [23]:
busan_3rd = busan_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [24]:
busan_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상영,득표수_2_새천년민주당_한이헌,득표수_3_민주노동당_김석준,득표수_계,무효투표수,기권수
0,부산광역시,중구,45490,23688,15669,4039,3487,23195,493,21802
40,부산광역시,서구,116858,47313,31336,8397,6594,46327,986,69545
113,부산광역시,동구,97322,44749,29853,7808,6247,43908,841,52573
173,부산광역시,영도구,136484,59273,37299,12417,8545,58261,1012,77211
244,부산광역시,부산진구,320729,152469,95573,28774,25497,149844,2625,168260


In [25]:
# 수치형 열만 합계 구하기
summary_row = busan_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '부산광역시')

# summary_row를 맨 위에 붙이기
busan_3rd_with_total = pd.concat([summary_row, busan_3rd], ignore_index=True)

In [26]:
busan_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상영,득표수_2_새천년민주당_한이헌,득표수_3_민주노동당_김석준,득표수_계,무효투표수,기권수
0,부산광역시,합계,2784721,1163040,729589,221938,192594,1144121,18919,1621681
1,부산광역시,중구,45490,23688,15669,4039,3487,23195,493,21802
2,부산광역시,서구,116858,47313,31336,8397,6594,46327,986,69545
3,부산광역시,동구,97322,44749,29853,7808,6247,43908,841,52573
4,부산광역시,영도구,136484,59273,37299,12417,8545,58261,1012,77211
5,부산광역시,부산진구,320729,152469,95573,28774,25497,149844,2625,168260
6,부산광역시,동래구,219090,92495,59940,15482,15783,91205,1290,126595
7,부산광역시,남구,230494,99050,62976,17767,16906,97649,1401,131444
8,부산광역시,북구,221411,82584,49151,18109,14128,81388,1196,138827
9,부산광역시,해운대구,289845,114749,69300,22401,21305,113006,1743,175096


In [27]:
busan_3rd_with_total.to_csv("temp1_governor_busan_3.csv", index=False, encoding="utf-8-sig")


## Daegu


In [28]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_daegu = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EB%8C%80%EA%B5%AC.xls'

daegu_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_daegu,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 810

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 조해녕
5: 무소속 이재용
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 810행 → 필터링 후: 8행
'투표구명' == '합계'인 행만 추출


In [29]:
daegu_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 조해녕,무소속 이재용,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(대구),합계,70409,34423,20807,13020,NaN,33827,48.9,98.3
45,동구(대구),합계,250562,105543,66098,37617,NaN,103715,42.1,98.3
155,서구(대구),합계,206981,84479,53305,29549,NaN,82854,40.8,98.1
253,남구(대구),합계,148146,64865,31249,32866,NaN,64115,43.8,98.8
320,북구(대구),합계,294071,117993,74706,41585,NaN,116291,40.1,98.6
445,수성구,합계,320476,134338,81824,50820,NaN,132644,41.9,98.7
581,달서구,합계,414225,155487,91557,61725,NaN,153282,37.5,98.6
737,달성군,합계,109408,54866,33397,20214,NaN,53611,50.1,97.7


In [30]:
daegu_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 조해녕',
 '무소속 이재용',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [31]:
rename_daegu = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 조해녕': '득표수_1_한나라당_조해녕',
    '무소속 이재용': '득표수_3_무소속_이재용',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [32]:
daegu_3rd = daegu_3rd.rename(columns=rename_daegu).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
daegu_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_조해녕,득표수_3_무소속_이재용,득표수_계
0,중구(대구),70409,34423,20807,13020,33827
45,동구(대구),250562,105543,66098,37617,103715
155,서구(대구),206981,84479,53305,29549,82854
253,남구(대구),148146,64865,31249,32866,64115
320,북구(대구),294071,117993,74706,41585,116291
445,수성구,320476,134338,81824,50820,132644
581,달서구,414225,155487,91557,61725,153282
737,달성군,109408,54866,33397,20214,53611


In [33]:
daegu_3rd = daegu_3rd.assign(
    시도='대구광역시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + daegu_3rd.columns.tolist()]

In [34]:
daegu_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_조해녕,득표수_3_무소속_이재용,득표수_계
0,대구광역시,중구,70409,34423,20807,13020,33827
45,대구광역시,동구,250562,105543,66098,37617,103715
155,대구광역시,서구,206981,84479,53305,29549,82854
253,대구광역시,남구,148146,64865,31249,32866,64115
320,대구광역시,북구,294071,117993,74706,41585,116291


In [35]:
daegu_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 737
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   시도              8 non-null      object
 1   구시군             8 non-null      object
 2   선거인수            8 non-null      object
 3   투표수             8 non-null      object
 4   득표수_1_한나라당_조해녕  8 non-null      object
 5   득표수_3_무소속_이재용   8 non-null      object
 6   득표수_계           8 non-null      object
dtypes: object(7)
memory usage: 812.0+ bytes


In [36]:
daegu_3rd = daegu_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [37]:
daegu_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_조해녕,득표수_3_무소속_이재용,득표수_계,무효투표수,기권수
0,대구광역시,중구,70409,34423,20807,13020,33827,596,35986
45,대구광역시,동구,250562,105543,66098,37617,103715,1828,145019
155,대구광역시,서구,206981,84479,53305,29549,82854,1625,122502
253,대구광역시,남구,148146,64865,31249,32866,64115,750,83281
320,대구광역시,북구,294071,117993,74706,41585,116291,1702,176078


In [38]:
# 수치형 열만 합계 구하기
summary_row = daegu_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대구광역시')

# summary_row를 맨 위에 붙이기
daegu_3rd_with_total = pd.concat([summary_row, daegu_3rd], ignore_index=True)

In [39]:
daegu_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_조해녕,득표수_3_무소속_이재용,득표수_계,무효투표수,기권수
0,대구광역시,합계,1814278,751994,452943,287396,740339,11655,1062284
1,대구광역시,중구,70409,34423,20807,13020,33827,596,35986
2,대구광역시,동구,250562,105543,66098,37617,103715,1828,145019
3,대구광역시,서구,206981,84479,53305,29549,82854,1625,122502
4,대구광역시,남구,148146,64865,31249,32866,64115,750,83281
5,대구광역시,북구,294071,117993,74706,41585,116291,1702,176078
6,대구광역시,수성구,320476,134338,81824,50820,132644,1694,186138
7,대구광역시,달서구,414225,155487,91557,61725,153282,2205,258738
8,대구광역시,달성군,109408,54866,33397,20214,53611,1255,54542


In [40]:
daegu_3rd_with_total.to_csv("temp1_governor_daegu_3.csv", index=False, encoding="utf-8-sig")


## Incheon


In [41]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_incheon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%9D%B8%EC%B2%9C.xls'

incheon_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_incheon,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 13
데이터 행 수: 814

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 안상수
5: 민주당 박상은
6: 녹색평화당 신맹순
7: 민주노동당 김창한
8: 사회당 김영규
9: _

'투표구명' 컬럼 발견: 투표구명
필터링 전: 814행 → 필터링 후: 10행
'투표구명' == '합계'인 행만 추출


In [42]:
incheon_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 안상수,민주당 박상은,녹색평화당 신맹순,민주노동당 김창한,사회당 김영규,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(인천),합계,56476,27197,15627,8295,1154,987,599,NaN,26662,48.2,98
45,동구(인천),합계,57370,28452,16071,8484,971,1484,806,NaN,27816,49.6,97.8
87,남구(인천),합계,317337,124830,71928,37404,5341,5028,3276,NaN,122977,39.3,98.5
213,연수구,합계,174043,67745,40332,19621,2711,2590,1681,NaN,66935,38.9,98.8
279,남동구,합계,286397,105918,56823,33644,6905,4372,2805,NaN,104549,37,98.7
391,부평구,합계,391526,148406,78925,47685,5728,10827,3440,NaN,146605,37.9,98.8
538,계양구,합계,226315,79673,45748,25930,2376,3144,1637,NaN,78835,35.2,98.9
619,서구(인천),합계,237302,85958,46138,27657,3132,5544,2252,NaN,84723,36.2,98.6
714,강화군,합계,51913,34626,17134,14150,755,936,679,NaN,33654,66.7,97.2
776,옹진군,합계,11228,8797,5206,2340,400,322,229,NaN,8497,78.3,96.6


In [43]:
incheon_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 안상수',
 '민주당 박상은',
 '녹색평화당 신맹순',
 '민주노동당 김창한',
 '사회당 김영규',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [44]:
rename_incheon = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 안상수': '득표수_1_한나라당_안상수',
    '민주당 박상은': '득표수_2_새천년민주당_박상은',
    '녹색평화당 신맹순': '득표수_3_녹색평화당_신맹순',
    '민주노동당 김창한': '득표수_4_민주노동당_김창한',
    '사회당 김영규': '득표수_5_사회당_김영규',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [45]:
incheon_3rd = incheon_3rd.rename(columns=rename_incheon).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
incheon_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_안상수,득표수_2_새천년민주당_박상은,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계
0,중구(인천),56476,27197,15627,8295,1154,987,599,26662
45,동구(인천),57370,28452,16071,8484,971,1484,806,27816
87,남구(인천),317337,124830,71928,37404,5341,5028,3276,122977
213,연수구,174043,67745,40332,19621,2711,2590,1681,66935
279,남동구,286397,105918,56823,33644,6905,4372,2805,104549
391,부평구,391526,148406,78925,47685,5728,10827,3440,146605
538,계양구,226315,79673,45748,25930,2376,3144,1637,78835
619,서구(인천),237302,85958,46138,27657,3132,5544,2252,84723
714,강화군,51913,34626,17134,14150,755,936,679,33654
776,옹진군,11228,8797,5206,2340,400,322,229,8497


In [46]:
incheon_3rd = incheon_3rd.assign(
    시도='인천광역시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + incheon_3rd.columns.tolist()]

In [47]:
incheon_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상수,득표수_2_새천년민주당_박상은,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계
0,인천광역시,중구,56476,27197,15627,8295,1154,987,599,26662
45,인천광역시,동구,57370,28452,16071,8484,971,1484,806,27816
87,인천광역시,남구,317337,124830,71928,37404,5341,5028,3276,122977
213,인천광역시,연수구,174043,67745,40332,19621,2711,2590,1681,66935
279,인천광역시,남동구,286397,105918,56823,33644,6905,4372,2805,104549


In [48]:
incheon_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 776
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                10 non-null     object
 1   구시군               10 non-null     object
 2   선거인수              10 non-null     object
 3   투표수               10 non-null     object
 4   득표수_1_한나라당_안상수    10 non-null     object
 5   득표수_2_새천년민주당_박상은  10 non-null     object
 6   득표수_3_녹색평화당_신맹순   10 non-null     object
 7   득표수_4_민주노동당_김창한   10 non-null     object
 8   득표수_5_사회당_김영규     10 non-null     object
 9   득표수_계             10 non-null     object
dtypes: object(10)
memory usage: 1.2+ KB


In [49]:
incheon_3rd = incheon_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [50]:
incheon_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상수,득표수_2_새천년민주당_박상은,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계,무효투표수,기권수
0,인천광역시,중구,56476,27197,15627,8295,1154,987,599,26662,535,29279
45,인천광역시,동구,57370,28452,16071,8484,971,1484,806,27816,636,28918
87,인천광역시,남구,317337,124830,71928,37404,5341,5028,3276,122977,1853,192507
213,인천광역시,연수구,174043,67745,40332,19621,2711,2590,1681,66935,810,106298
279,인천광역시,남동구,286397,105918,56823,33644,6905,4372,2805,104549,1369,180479


In [51]:
# 수치형 열만 합계 구하기
summary_row = incheon_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '인천광역시')

# summary_row를 맨 위에 붙이기
incheon_3rd_with_total = pd.concat([summary_row, incheon_3rd], ignore_index=True)

In [52]:
incheon_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상수,득표수_2_새천년민주당_박상은,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계,무효투표수,기권수
0,인천광역시,합계,1809907,711602,393932,225210,29473,35234,17404,701253,10349,1098305
1,인천광역시,중구,56476,27197,15627,8295,1154,987,599,26662,535,29279
2,인천광역시,동구,57370,28452,16071,8484,971,1484,806,27816,636,28918
3,인천광역시,남구,317337,124830,71928,37404,5341,5028,3276,122977,1853,192507
4,인천광역시,연수구,174043,67745,40332,19621,2711,2590,1681,66935,810,106298
5,인천광역시,남동구,286397,105918,56823,33644,6905,4372,2805,104549,1369,180479
6,인천광역시,부평구,391526,148406,78925,47685,5728,10827,3440,146605,1801,243120
7,인천광역시,계양구,226315,79673,45748,25930,2376,3144,1637,78835,838,146642
8,인천광역시,서구,237302,85958,46138,27657,3132,5544,2252,84723,1235,151344
9,인천광역시,강화군,51913,34626,17134,14150,755,936,679,33654,972,17287


In [53]:
incheon_3rd_with_total.to_csv("temp1_governor_incheon_3.csv", index=False, encoding="utf-8-sig")


## Gwangju


In [54]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gwangju = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B4%91%EC%A3%BC.xls'

gwangju_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gwangju,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 14
데이터 행 수: 465

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이환의
5: 민주당 박광태
6: 민주노동당 박종현
7: 무소속 정구선
8: 무소속 정동년
9: 무소속 정호선

'투표구명' 컬럼 발견: 투표구명
필터링 전: 465행 → 필터링 후: 5행
'투표구명' == '합계'인 행만 추출


In [55]:
gwangju_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이환의,민주당 박광태,민주노동당 박종현,무소속 정구선,무소속 정동년,무소속 정호선,_,계,투 표 율 (%),유효 투표 율 (%)
0,동구(광주),합계,89770,42924,6436,20363,2593,1171,9445,2144,NaN,42152,47.8,98.2
58,서구(광주),합계,204779,85482,10740,39589,5565,1778,22435,4219,NaN,84326,41.7,98.6
148,남구(광주),합계,159676,71438,7523,34966,3652,1647,18855,3679,NaN,70322,44.7,98.4
231,북구(광주),합계,324452,128528,12760,55377,9123,3568,38680,7156,NaN,126664,39.6,98.5
367,광산구,합계,175804,75449,6236,35643,8494,1713,18000,3598,NaN,73684,42.9,97.7


In [56]:
gwangju_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 이환의',
 '민주당 박광태',
 '민주노동당 박종현',
 '무소속 정구선',
 '무소속 정동년',
 '무소속 정호선',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [57]:
rename_gwangju = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 이환의': '득표수_1_한나라당_이환의',
    '민주당 박광태': '득표수_2_새천년민주당_박광태',
    '민주노동당 박종현': '득표수_3_민주노동당_박종현',
    '무소속 정구선': '득표수_4_무소속_정구선',
    '무소속 정동년': '득표수_5_무소속_정동년',
    '무소속 정호선': '득표수_6_무소속_정호선',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [58]:
gwangju_3rd = gwangju_3rd.rename(columns=rename_gwangju).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
gwangju_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_이환의,득표수_2_새천년민주당_박광태,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계
0,동구(광주),89770,42924,6436,20363,2593,1171,9445,2144,42152
58,서구(광주),204779,85482,10740,39589,5565,1778,22435,4219,84326
148,남구(광주),159676,71438,7523,34966,3652,1647,18855,3679,70322
231,북구(광주),324452,128528,12760,55377,9123,3568,38680,7156,126664
367,광산구,175804,75449,6236,35643,8494,1713,18000,3598,73684


In [59]:
gwangju_3rd = gwangju_3rd.assign(
    시도='광주광역시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + gwangju_3rd.columns.tolist()]

In [60]:
gwangju_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이환의,득표수_2_새천년민주당_박광태,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계
0,광주광역시,동구,89770,42924,6436,20363,2593,1171,9445,2144,42152
58,광주광역시,서구,204779,85482,10740,39589,5565,1778,22435,4219,84326
148,광주광역시,남구,159676,71438,7523,34966,3652,1647,18855,3679,70322
231,광주광역시,북구,324452,128528,12760,55377,9123,3568,38680,7156,126664
367,광주광역시,광산구,175804,75449,6236,35643,8494,1713,18000,3598,73684


In [61]:
gwangju_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 367
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                5 non-null      object
 1   구시군               5 non-null      object
 2   선거인수              5 non-null      object
 3   투표수               5 non-null      object
 4   득표수_1_한나라당_이환의    5 non-null      object
 5   득표수_2_새천년민주당_박광태  5 non-null      object
 6   득표수_3_민주노동당_박종현   5 non-null      object
 7   득표수_4_무소속_정구선     5 non-null      object
 8   득표수_5_무소속_정동년     5 non-null      object
 9   득표수_6_무소속_정호선     5 non-null      object
 10  득표수_계             5 non-null      object
dtypes: object(11)
memory usage: 652.0+ bytes


In [62]:
gwangju_3rd = gwangju_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [63]:
gwangju_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이환의,득표수_2_새천년민주당_박광태,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계,무효투표수,기권수
0,광주광역시,동구,89770,42924,6436,20363,2593,1171,9445,2144,42152,772,46846
58,광주광역시,서구,204779,85482,10740,39589,5565,1778,22435,4219,84326,1156,119297
148,광주광역시,남구,159676,71438,7523,34966,3652,1647,18855,3679,70322,1116,88238
231,광주광역시,북구,324452,128528,12760,55377,9123,3568,38680,7156,126664,1864,195924
367,광주광역시,광산구,175804,75449,6236,35643,8494,1713,18000,3598,73684,1765,100355


In [64]:
# 수치형 열만 합계 구하기
summary_row = gwangju_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '광주광역시')

# summary_row를 맨 위에 붙이기
gwangju_3rd_with_total = pd.concat([summary_row, gwangju_3rd], ignore_index=True)

In [65]:
gwangju_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이환의,득표수_2_새천년민주당_박광태,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계,무효투표수,기권수
0,광주광역시,합계,954481,403821,43695,185938,29427,9877,107415,20796,397148,6673,550660
1,광주광역시,동구,89770,42924,6436,20363,2593,1171,9445,2144,42152,772,46846
2,광주광역시,서구,204779,85482,10740,39589,5565,1778,22435,4219,84326,1156,119297
3,광주광역시,남구,159676,71438,7523,34966,3652,1647,18855,3679,70322,1116,88238
4,광주광역시,북구,324452,128528,12760,55377,9123,3568,38680,7156,126664,1864,195924
5,광주광역시,광산구,175804,75449,6236,35643,8494,1713,18000,3598,73684,1765,100355


In [66]:
gwangju_3rd_with_total.to_csv("temp1_governor_gwangju_3.csv", index=False, encoding="utf-8-sig")


## Daejeon


In [67]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_daejeon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EB%8C%80%EC%A0%84.xls'

daejeon_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_daejeon,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 12
데이터 행 수: 440

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 염홍철
5: 자민련 홍선기
6: 무소속 김헌태
7: 무소속 정하용
8: _
9: 계

'투표구명' 컬럼 발견: 투표구명
필터링 전: 440행 → 필터링 후: 5행
'투표구명' == '합계'인 행만 추출


In [68]:
daejeon_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 염홍철,자민련 홍선기,무소속 김헌태,무소속 정하용,_,계,투 표 율 (%),유효 투표 율 (%)
0,동구(대전),합계,181210,76220,32969,32035,3066,6623,NaN,74693,42.1,98
90,중구(대전),합계,195396,83818,37980,33683,3105,7742,NaN,82510,42.9,98.4
183,서구(대전),합계,333834,140783,67939,53426,4711,13131,NaN,139207,42.2,98.9
311,유성구,합계,119056,52548,24855,19643,2112,5102,NaN,51712,44.1,98.4
366,대덕구,합계,157684,64440,28089,26639,2786,5847,NaN,63361,40.9,98.3


In [69]:
daejeon_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 염홍철',
 '자민련 홍선기',
 '무소속 김헌태',
 '무소속 정하용',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [70]:
rename_daejeon = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 염홍철': '득표수_1_한나라당_염홍철',
    '자민련 홍선기': '득표수_3_자유민주연합_홍선기',
    '무소속 김헌태': '득표수_4_무소속_김헌태',
    '무소속 정하용': '득표수_5_무소속_정하용',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [71]:
daejeon_3rd = daejeon_3rd.rename(columns=rename_daejeon).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
daejeon_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_염홍철,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계
0,동구(대전),181210,76220,32969,32035,3066,6623,74693
90,중구(대전),195396,83818,37980,33683,3105,7742,82510
183,서구(대전),333834,140783,67939,53426,4711,13131,139207
311,유성구,119056,52548,24855,19643,2112,5102,51712
366,대덕구,157684,64440,28089,26639,2786,5847,63361


In [72]:
daejeon_3rd = daejeon_3rd.assign(
    시도='대전광역시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + daejeon_3rd.columns.tolist()]

In [73]:
daejeon_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_염홍철,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계
0,대전광역시,동구,181210,76220,32969,32035,3066,6623,74693
90,대전광역시,중구,195396,83818,37980,33683,3105,7742,82510
183,대전광역시,서구,333834,140783,67939,53426,4711,13131,139207
311,대전광역시,유성구,119056,52548,24855,19643,2112,5102,51712
366,대전광역시,대덕구,157684,64440,28089,26639,2786,5847,63361


In [74]:
daejeon_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 366
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                5 non-null      object
 1   구시군               5 non-null      object
 2   선거인수              5 non-null      object
 3   투표수               5 non-null      object
 4   득표수_1_한나라당_염홍철    5 non-null      object
 5   득표수_3_자유민주연합_홍선기  5 non-null      object
 6   득표수_4_무소속_김헌태     5 non-null      object
 7   득표수_5_무소속_정하용     5 non-null      object
 8   득표수_계             5 non-null      object
dtypes: object(9)
memory usage: 572.0+ bytes


In [75]:
daejeon_3rd = daejeon_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [76]:
daejeon_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_염홍철,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계,무효투표수,기권수
0,대전광역시,동구,181210,76220,32969,32035,3066,6623,74693,1527,104990
90,대전광역시,중구,195396,83818,37980,33683,3105,7742,82510,1308,111578
183,대전광역시,서구,333834,140783,67939,53426,4711,13131,139207,1576,193051
311,대전광역시,유성구,119056,52548,24855,19643,2112,5102,51712,836,66508
366,대전광역시,대덕구,157684,64440,28089,26639,2786,5847,63361,1079,93244


In [77]:
# 수치형 열만 합계 구하기
summary_row = daejeon_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대전광역시')

# summary_row를 맨 위에 붙이기
daejeon_3rd_with_total = pd.concat([summary_row, daejeon_3rd], ignore_index=True)

In [78]:
daejeon_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_염홍철,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계,무효투표수,기권수
0,대전광역시,합계,987180,417809,191832,165426,15780,38445,411483,6326,569371
1,대전광역시,동구,181210,76220,32969,32035,3066,6623,74693,1527,104990
2,대전광역시,중구,195396,83818,37980,33683,3105,7742,82510,1308,111578
3,대전광역시,서구,333834,140783,67939,53426,4711,13131,139207,1576,193051
4,대전광역시,유성구,119056,52548,24855,19643,2112,5102,51712,836,66508
5,대전광역시,대덕구,157684,64440,28089,26639,2786,5847,63361,1079,93244


In [79]:
daejeon_3rd_with_total.to_csv("temp1_governor_daejeon_3.csv", index=False, encoding="utf-8-sig")


## Ulsan


In [80]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_ulsan = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%9A%B8%EC%82%B0.xls'

ulsan_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_ulsan,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 372

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 박맹우
5: 민주노동당 송철호
6: 사회당 안승천
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 372행 → 필터링 후: 5행
'투표구명' == '합계'인 행만 추출


In [81]:
ulsan_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 박맹우,민주노동당 송철호,사회당 안승천,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(울산),합계,162972,81207,45410,32569,2113,NaN,80092,49.8,98.6
77,남구(울산),합계,234702,108730,63805,40808,2996,NaN,107609,46.3,99
176,동구(울산),합계,125305,70935,28200,39071,2693,NaN,69964,56.6,98.6
235,북구(울산),합계,79943,47783,20174,25593,1424,NaN,47191,59.8,98.8
285,울주군,합계,119884,69466,40183,24505,3103,NaN,67791,57.9,97.6


In [82]:
ulsan_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 박맹우',
 '민주노동당 송철호',
 '사회당 안승천',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [83]:
rename_ulsan = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 박맹우': '득표수_1_한나라당_박맹우',
    '민주노동당 송철호': '득표수_3_민주노동당_송철호',
    '사회당 안승천': '득표수_4_사회당_안승천',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [84]:
ulsan_3rd = ulsan_3rd.rename(columns=rename_ulsan).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
ulsan_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_박맹우,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계
0,중구(울산),162972,81207,45410,32569,2113,80092
77,남구(울산),234702,108730,63805,40808,2996,107609
176,동구(울산),125305,70935,28200,39071,2693,69964
235,북구(울산),79943,47783,20174,25593,1424,47191
285,울주군,119884,69466,40183,24505,3103,67791


In [85]:
ulsan_3rd = ulsan_3rd.assign(
    시도='울산광역시',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + ulsan_3rd.columns.tolist()]

In [86]:
ulsan_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박맹우,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계
0,울산광역시,중구,162972,81207,45410,32569,2113,80092
77,울산광역시,남구,234702,108730,63805,40808,2996,107609
176,울산광역시,동구,125305,70935,28200,39071,2693,69964
235,울산광역시,북구,79943,47783,20174,25593,1424,47191
285,울산광역시,울주군,119884,69466,40183,24505,3103,67791


In [87]:
ulsan_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 285
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               5 non-null      object
 1   구시군              5 non-null      object
 2   선거인수             5 non-null      object
 3   투표수              5 non-null      object
 4   득표수_1_한나라당_박맹우   5 non-null      object
 5   득표수_3_민주노동당_송철호  5 non-null      object
 6   득표수_4_사회당_안승천    5 non-null      object
 7   득표수_계            5 non-null      object
dtypes: object(8)
memory usage: 532.0+ bytes


In [88]:
ulsan_3rd = ulsan_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [89]:
ulsan_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박맹우,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계,무효투표수,기권수
0,울산광역시,중구,162972,81207,45410,32569,2113,80092,1115,81765
77,울산광역시,남구,234702,108730,63805,40808,2996,107609,1121,125972
176,울산광역시,동구,125305,70935,28200,39071,2693,69964,971,54370
235,울산광역시,북구,79943,47783,20174,25593,1424,47191,592,32160
285,울산광역시,울주군,119884,69466,40183,24505,3103,67791,1675,50418


In [90]:
# 수치형 열만 합계 구하기
summary_row = ulsan_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '울산광역시')

# summary_row를 맨 위에 붙이기
ulsan_3rd_with_total = pd.concat([summary_row, ulsan_3rd], ignore_index=True)

In [91]:
ulsan_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박맹우,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계,무효투표수,기권수
0,울산광역시,합계,722806,378121,197772,162546,12329,372647,5474,344685
1,울산광역시,중구,162972,81207,45410,32569,2113,80092,1115,81765
2,울산광역시,남구,234702,108730,63805,40808,2996,107609,1121,125972
3,울산광역시,동구,125305,70935,28200,39071,2693,69964,971,54370
4,울산광역시,북구,79943,47783,20174,25593,1424,47191,592,32160
5,울산광역시,울주군,119884,69466,40183,24505,3103,67791,1675,50418


In [92]:
ulsan_3rd_with_total.to_csv("temp1_governor_ulsan_3.csv", index=False, encoding="utf-8-sig")


## Gyeonggi


In [93]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gyeonggi = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B2%BD%EA%B8%B0.xls'

gyeonggi_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gyeonggi,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 3231

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 손학규
5: 민주당 진념
6: 민주노동당 김준기
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 3231행 → 필터링 후: 39행
'투표구명' == '합계'인 행만 추출


In [94]:
gyeonggi_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 손학규,민주당 진념,민주노동당 김준기,_,계,투 표 율 (%),유효 투표 율 (%)
0,수원시장안구,합계,233829,99253,59603,33422,5251,NaN,98276,42.4,99
90,수원시권선구,합계,225950,92598,55604,31178,4856,NaN,91638,41,99
180,수원시팔달구,합계,224122,88058,53550,29845,3859,NaN,87254,39.3,99.1
260,성남시수정구,합계,192566,78431,37715,34523,5176,NaN,77414,40.7,98.7
343,성남시중원구,합계,199094,76103,36501,33577,4923,NaN,75001,38.2,98.6
418,성남시분당구,합계,281390,135540,86423,44447,3846,NaN,134716,48.2,99.4
533,의정부시,합계,260157,110516,61852,40860,6429,NaN,109141,42.5,98.8
637,안양시만안구,합계,191427,78621,44315,29802,3736,NaN,77853,41.1,99
713,안양시동안구,합계,224569,104102,60990,38365,3936,NaN,103291,46.4,99.2
808,부천시원미구,합계,275354,112168,60252,45344,5443,NaN,111039,40.7,99


In [95]:
gyeonggi_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 손학규',
 '민주당 진념',
 '민주노동당 김준기',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [96]:
rename_gyeonggi = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 손학규': '득표수_1_한나라당_손학규',
    '민주당 진념': '득표수_2_새천년민주당_진념',
    '민주노동당 김준기': '득표수_3_민주노동당_김준기',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [97]:
gyeonggi_3rd = gyeonggi_3rd.rename(columns=rename_gyeonggi).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
gyeonggi_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_손학규,득표수_2_새천년민주당_진념,득표수_3_민주노동당_김준기,득표수_계
0,수원시장안구,233829,99253,59603,33422,5251,98276
90,수원시권선구,225950,92598,55604,31178,4856,91638
180,수원시팔달구,224122,88058,53550,29845,3859,87254
260,성남시수정구,192566,78431,37715,34523,5176,77414
343,성남시중원구,199094,76103,36501,33577,4923,75001
418,성남시분당구,281390,135540,86423,44447,3846,134716
533,의정부시,260157,110516,61852,40860,6429,109141
637,안양시만안구,191427,78621,44315,29802,3736,77853
713,안양시동안구,224569,104102,60990,38365,3936,103291
808,부천시원미구,275354,112168,60252,45344,5443,111039


In [98]:
gyeonggi_3rd = gyeonggi_3rd.assign(
    시도='경기도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + gyeonggi_3rd.columns.tolist()]

In [99]:
gyeonggi_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_손학규,득표수_2_새천년민주당_진념,득표수_3_민주노동당_김준기,득표수_계
0,경기도,수원시장안구,233829,99253,59603,33422,5251,98276
90,경기도,수원시권선구,225950,92598,55604,31178,4856,91638
180,경기도,수원시팔달구,224122,88058,53550,29845,3859,87254
260,경기도,성남시수정구,192566,78431,37715,34523,5176,77414
343,경기도,성남시중원구,199094,76103,36501,33577,4923,75001


In [100]:
gyeonggi_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 3184
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               39 non-null     object
 1   구시군              39 non-null     object
 2   선거인수             39 non-null     object
 3   투표수              39 non-null     object
 4   득표수_1_한나라당_손학규   39 non-null     object
 5   득표수_2_새천년민주당_진념  39 non-null     object
 6   득표수_3_민주노동당_김준기  39 non-null     object
 7   득표수_계            39 non-null     object
dtypes: object(8)
memory usage: 3.8+ KB


In [101]:
gyeonggi_3rd = gyeonggi_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [102]:
gyeonggi_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_손학규,득표수_2_새천년민주당_진념,득표수_3_민주노동당_김준기,득표수_계,무효투표수,기권수
0,경기도,수원시장안구,233829,99253,59603,33422,5251,98276,977,134576
90,경기도,수원시권선구,225950,92598,55604,31178,4856,91638,960,133352
180,경기도,수원시팔달구,224122,88058,53550,29845,3859,87254,804,136064
260,경기도,성남시수정구,192566,78431,37715,34523,5176,77414,1017,114135
343,경기도,성남시중원구,199094,76103,36501,33577,4923,75001,1102,122991


In [103]:
# 수치형 열만 합계 구하기
summary_row = gyeonggi_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경기도')

# summary_row를 맨 위에 붙이기
gyeonggi_3rd_with_total = pd.concat([summary_row, gyeonggi_3rd], ignore_index=True)

In [104]:
gyeonggi_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_손학규,득표수_2_새천년민주당_진념,득표수_3_민주노동당_김준기,득표수_계,무효투표수,기권수
0,경기도,합계,6777575,3024844,1744291,1075243,168357,2987891,36953,3752731
1,경기도,수원시장안구,233829,99253,59603,33422,5251,98276,977,134576
2,경기도,수원시권선구,225950,92598,55604,31178,4856,91638,960,133352
3,경기도,수원시팔달구,224122,88058,53550,29845,3859,87254,804,136064
4,경기도,성남시수정구,192566,78431,37715,34523,5176,77414,1017,114135
5,경기도,성남시중원구,199094,76103,36501,33577,4923,75001,1102,122991
6,경기도,성남시분당구,281390,135540,86423,44447,3846,134716,824,145850
7,경기도,의정부시,260157,110516,61852,40860,6429,109141,1375,149641
8,경기도,안양시만안구,191427,78621,44315,29802,3736,77853,768,112806
9,경기도,안양시동안구,224569,104102,60990,38365,3936,103291,811,120467


In [105]:
gyeonggi_3rd_with_total.to_csv("temp1_governor_gyeonggi_3.csv", index=False, encoding="utf-8-sig")


## Gangwon


In [106]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gangwon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B0%95%EC%9B%90.xls'

gangwon_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gangwon,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 1073

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 김진선
5: 민주당 남동우
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1073행 → 필터링 후: 18행
'투표구명' == '합계'인 행만 추출


In [107]:
gangwon_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 김진선,민주당 남동우,_,계,투 표 율 (%),유효 투표 율 (%)
0,춘천시,합계,181938,95277,52497,41426,NaN,93923,52.4,98.6
120,원주시,합계,194677,97205,62405,32931,NaN,95336,49.9,98.1
240,강릉시,합계,169090,91290,72430,17571,NaN,90001,54,98.6
357,동해시,합계,74930,42530,35744,6107,NaN,41851,56.8,98.4
409,삼척시,합계,59541,41665,33307,7553,NaN,40860,70,98.1
490,태백시,합계,41864,25876,20400,5075,NaN,25475,61.8,98.5
531,정선군,합계,37031,26016,19524,5873,NaN,25397,70.3,97.6
585,속초시,합계,64193,36439,26621,9175,NaN,35796,56.8,98.2
627,고성군(강원),합계,26309,19349,14401,4490,NaN,18891,73.5,97.6
660,양양군,합계,23171,17835,13285,4093,NaN,17378,77,97.4


In [108]:
gangwon_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 김진선',
 '민주당 남동우',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [109]:
rename_gangwon = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 김진선': '득표수_1_한나라당_김진선',
    '민주당 남동우': '득표수_2_새천년민주당_남동우',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [110]:
gangwon_3rd = gangwon_3rd.rename(columns=rename_gangwon).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
gangwon_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_김진선,득표수_2_새천년민주당_남동우,득표수_계
0,춘천시,181938,95277,52497,41426,93923
120,원주시,194677,97205,62405,32931,95336
240,강릉시,169090,91290,72430,17571,90001
357,동해시,74930,42530,35744,6107,41851
409,삼척시,59541,41665,33307,7553,40860
490,태백시,41864,25876,20400,5075,25475
531,정선군,37031,26016,19524,5873,25397
585,속초시,64193,36439,26621,9175,35796
627,고성군(강원),26309,19349,14401,4490,18891
660,양양군,23171,17835,13285,4093,17378


In [111]:
gangwon_3rd = gangwon_3rd.assign(
    시도='강원도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + gangwon_3rd.columns.tolist()]

In [112]:
gangwon_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김진선,득표수_2_새천년민주당_남동우,득표수_계
0,강원도,춘천시,181938,95277,52497,41426,93923
120,강원도,원주시,194677,97205,62405,32931,95336
240,강원도,강릉시,169090,91290,72430,17571,90001
357,강원도,동해시,74930,42530,35744,6107,41851
409,강원도,삼척시,59541,41665,33307,7553,40860


In [113]:
gangwon_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 0 to 1025
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                18 non-null     object
 1   구시군               18 non-null     object
 2   선거인수              18 non-null     object
 3   투표수               18 non-null     object
 4   득표수_1_한나라당_김진선    18 non-null     object
 5   득표수_2_새천년민주당_남동우  18 non-null     object
 6   득표수_계             18 non-null     object
dtypes: object(7)
memory usage: 1.7+ KB


In [114]:
gangwon_3rd = gangwon_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [115]:
gangwon_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김진선,득표수_2_새천년민주당_남동우,득표수_계,무효투표수,기권수
0,강원도,춘천시,181938,95277,52497,41426,93923,1354,86661
120,강원도,원주시,194677,97205,62405,32931,95336,1869,97472
240,강원도,강릉시,169090,91290,72430,17571,90001,1289,77800
357,강원도,동해시,74930,42530,35744,6107,41851,679,32400
409,강원도,삼척시,59541,41665,33307,7553,40860,805,17876


In [116]:
# 수치형 열만 합계 구하기
summary_row = gangwon_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '강원도')

# summary_row를 맨 위에 붙이기
gangwon_3rd_with_total = pd.concat([summary_row, gangwon_3rd], ignore_index=True)

In [117]:
gangwon_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김진선,득표수_2_새천년민주당_남동우,득표수_계,무효투표수,기권수
0,강원도,합계,1129859,671242,468987,190451,659438,11804,458617
1,강원도,춘천시,181938,95277,52497,41426,93923,1354,86661
2,강원도,원주시,194677,97205,62405,32931,95336,1869,97472
3,강원도,강릉시,169090,91290,72430,17571,90001,1289,77800
4,강원도,동해시,74930,42530,35744,6107,41851,679,32400
5,강원도,삼척시,59541,41665,33307,7553,40860,805,17876
6,강원도,태백시,41864,25876,20400,5075,25475,401,15988
7,강원도,정선군,37031,26016,19524,5873,25397,619,11015
8,강원도,속초시,64193,36439,26621,9175,35796,643,27754
9,강원도,고성군,26309,19349,14401,4490,18891,458,6960


In [118]:
gangwon_3rd_with_total.to_csv("temp1_governor_gangwon_3.csv", index=False, encoding="utf-8-sig")


## Chungbuk


In [119]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_chungbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%B6%A9%EB%B6%81.xls'

chungbuk_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_chungbuk,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 742

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이원종
5: 자민련 구천서
6: 무소속 장한량
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 742행 → 필터링 후: 12행
'투표구명' == '합계'인 행만 추출


In [120]:
chungbuk_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이원종,자민련 구천서,무소속 장한량,_,계,투 표 율 (%),유효 투표 율 (%)
0,청주시상당구,합계,163686,76240,46565,21875,6693,NaN,75133,46.6,98.5
67,청주시흥덕구,합계,246912,109497,66724,32496,8636,NaN,107856,44.3,98.5
173,충주시,합계,154380,83817,49029,26735,5968,NaN,81732,54.3,97.5
284,제천시,합계,105294,62841,46428,11768,3476,NaN,61672,59.7,98.1
364,단양군,합계,29210,20330,13104,5372,1310,NaN,19786,69.6,97.3
396,청원군,합계,92158,55498,28133,20965,4846,NaN,53944,60.2,97.2
467,영동군,합계,43630,31206,18517,9041,2699,NaN,30257,71.5,97
516,보은군,합계,32369,24757,8565,13877,1566,NaN,24008,76.5,97
566,옥천군,합계,45404,31654,12864,14975,2615,NaN,30454,69.7,96.2
607,음성군,합계,63645,41437,22367,14314,3562,NaN,40243,65.1,97.1


In [121]:
chungbuk_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 이원종',
 '자민련 구천서',
 '무소속 장한량',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [122]:
rename_chungbuk = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 이원종': '득표수_1_한나라당_이원종',
    '자민련 구천서': '득표수_3_자유민주연합_구천서',
    '무소속 장한량': '득표수_4_무소속_장한량',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [123]:
chungbuk_3rd = chungbuk_3rd.rename(columns=rename_chungbuk).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
chungbuk_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_이원종,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계
0,청주시상당구,163686,76240,46565,21875,6693,75133
67,청주시흥덕구,246912,109497,66724,32496,8636,107856
173,충주시,154380,83817,49029,26735,5968,81732
284,제천시,105294,62841,46428,11768,3476,61672
364,단양군,29210,20330,13104,5372,1310,19786
396,청원군,92158,55498,28133,20965,4846,53944
467,영동군,43630,31206,18517,9041,2699,30257
516,보은군,32369,24757,8565,13877,1566,24008
566,옥천군,45404,31654,12864,14975,2615,30454
607,음성군,63645,41437,22367,14314,3562,40243


In [124]:
chungbuk_3rd = chungbuk_3rd.assign(
    시도='충청북도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + chungbuk_3rd.columns.tolist()]

In [125]:
chungbuk_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이원종,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계
0,충청북도,청주시상당구,163686,76240,46565,21875,6693,75133
67,충청북도,청주시흥덕구,246912,109497,66724,32496,8636,107856
173,충청북도,충주시,154380,83817,49029,26735,5968,81732
284,충청북도,제천시,105294,62841,46428,11768,3476,61672
364,충청북도,단양군,29210,20330,13104,5372,1310,19786


In [126]:
chungbuk_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 0 to 689
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                12 non-null     object
 1   구시군               12 non-null     object
 2   선거인수              12 non-null     object
 3   투표수               12 non-null     object
 4   득표수_1_한나라당_이원종    12 non-null     object
 5   득표수_3_자유민주연합_구천서  12 non-null     object
 6   득표수_4_무소속_장한량     12 non-null     object
 7   득표수_계             12 non-null     object
dtypes: object(8)
memory usage: 1.1+ KB


In [127]:
chungbuk_3rd = chungbuk_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [128]:
chungbuk_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이원종,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계,무효투표수,기권수
0,충청북도,청주시상당구,163686,76240,46565,21875,6693,75133,1107,87446
67,충청북도,청주시흥덕구,246912,109497,66724,32496,8636,107856,1641,137415
173,충청북도,충주시,154380,83817,49029,26735,5968,81732,2085,70563
284,충청북도,제천시,105294,62841,46428,11768,3476,61672,1169,42453
364,충청북도,단양군,29210,20330,13104,5372,1310,19786,544,8880


In [129]:
# 수치형 열만 합계 구하기
summary_row = chungbuk_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청북도')

# summary_row를 맨 위에 붙이기
chungbuk_3rd_with_total = pd.concat([summary_row, chungbuk_3rd], ignore_index=True)

In [130]:
chungbuk_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이원종,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계,무효투표수,기권수
0,충청북도,합계,1076451,600550,343546,196253,46459,586258,14292,475901
1,충청북도,청주시상당구,163686,76240,46565,21875,6693,75133,1107,87446
2,충청북도,청주시흥덕구,246912,109497,66724,32496,8636,107856,1641,137415
3,충청북도,충주시,154380,83817,49029,26735,5968,81732,2085,70563
4,충청북도,제천시,105294,62841,46428,11768,3476,61672,1169,42453
5,충청북도,단양군,29210,20330,13104,5372,1310,19786,544,8880
6,충청북도,청원군,92158,55498,28133,20965,4846,53944,1554,36660
7,충청북도,영동군,43630,31206,18517,9041,2699,30257,949,12424
8,충청북도,보은군,32369,24757,8565,13877,1566,24008,749,7612
9,충청북도,옥천군,45404,31654,12864,14975,2615,30454,1200,13750


In [131]:
chungbuk_3rd_with_total.to_csv("temp1_governor_chungbuk_3.csv", index=False, encoding="utf-8-sig")


## Chungnam


In [132]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_chungnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%B6%A9%EB%82%A8.xls'

chungnam_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_chungnam,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 1071

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 박태권
5: 자민련 심대평
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1071행 → 필터링 후: 15행
'투표구명' == '합계'인 행만 추출


In [133]:
chungnam_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 박태권,자민련 심대평,_,계,투 표 율 (%),유효 투표 율 (%)
0,천안시,합계,299432,126601,48169,74741,NaN,122910,42.3,97.1
165,공주시,합계,98876,63126,13237,48033,NaN,61270,63.8,97.1
255,보령시,합계,85349,54816,14094,39142,NaN,53236,64.2,97.1
333,아산시,합계,135721,67878,22147,43398,NaN,65545,50,96.6
424,금산군,합계,48654,31078,7740,21909,NaN,29649,63.9,95.4
471,연기군,합계,60903,36324,9793,25493,NaN,35286,59.6,97.1
517,논산시,합계,122118,60890,15016,43755,NaN,58771,49.9,96.5
601,부여군,합계,68327,46428,9130,35761,NaN,44891,67.9,96.7
669,서천군,합계,55805,37802,10536,25308,NaN,35844,67.7,94.8
721,홍성군,합계,69735,44988,15993,27266,NaN,43259,64.5,96.2


In [134]:
chungnam_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 박태권',
 '자민련 심대평',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [135]:
rename_chungnam = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 박태권': '득표수_1_한나라당_박태권',
    '자민련 심대평': '득표수_3_자유민주연합_심대평',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [136]:
chungnam_3rd = chungnam_3rd.rename(columns=rename_chungnam).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
chungnam_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_박태권,득표수_3_자유민주연합_심대평,득표수_계
0,천안시,299432,126601,48169,74741,122910
165,공주시,98876,63126,13237,48033,61270
255,보령시,85349,54816,14094,39142,53236
333,아산시,135721,67878,22147,43398,65545
424,금산군,48654,31078,7740,21909,29649
471,연기군,60903,36324,9793,25493,35286
517,논산시,122118,60890,15016,43755,58771
601,부여군,68327,46428,9130,35761,44891
669,서천군,55805,37802,10536,25308,35844
721,홍성군,69735,44988,15993,27266,43259


In [137]:
chungnam_3rd = chungnam_3rd.assign(
    시도='충청남도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + chungnam_3rd.columns.tolist()]

In [138]:
chungnam_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박태권,득표수_3_자유민주연합_심대평,득표수_계
0,충청남도,천안시,299432,126601,48169,74741,122910
165,충청남도,공주시,98876,63126,13237,48033,61270
255,충청남도,보령시,85349,54816,14094,39142,53236
333,충청남도,아산시,135721,67878,22147,43398,65545
424,충청남도,금산군,48654,31078,7740,21909,29649


In [139]:
chungnam_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 1005
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                15 non-null     object
 1   구시군               15 non-null     object
 2   선거인수              15 non-null     object
 3   투표수               15 non-null     object
 4   득표수_1_한나라당_박태권    15 non-null     object
 5   득표수_3_자유민주연합_심대평  15 non-null     object
 6   득표수_계             15 non-null     object
dtypes: object(7)
memory usage: 1.5+ KB


In [140]:
chungnam_3rd = chungnam_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [141]:
chungnam_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박태권,득표수_3_자유민주연합_심대평,득표수_계,무효투표수,기권수
0,충청남도,천안시,299432,126601,48169,74741,122910,3691,172831
165,충청남도,공주시,98876,63126,13237,48033,61270,1856,35750
255,충청남도,보령시,85349,54816,14094,39142,53236,1580,30533
333,충청남도,아산시,135721,67878,22147,43398,65545,2333,67843
424,충청남도,금산군,48654,31078,7740,21909,29649,1429,17576


In [142]:
# 수치형 열만 합계 구하기
summary_row = chungnam_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청남도')

# summary_row를 맨 위에 붙이기
chungnam_3rd_with_total = pd.concat([summary_row, chungnam_3rd], ignore_index=True)

In [143]:
chungnam_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박태권,득표수_3_자유민주연합_심대평,득표수_계,무효투표수,기권수
0,충청남도,합계,1397105,785742,251005,508796,759801,25941,611363
1,충청남도,천안시,299432,126601,48169,74741,122910,3691,172831
2,충청남도,공주시,98876,63126,13237,48033,61270,1856,35750
3,충청남도,보령시,85349,54816,14094,39142,53236,1580,30533
4,충청남도,아산시,135721,67878,22147,43398,65545,2333,67843
5,충청남도,금산군,48654,31078,7740,21909,29649,1429,17576
6,충청남도,연기군,60903,36324,9793,25493,35286,1038,24579
7,충청남도,논산시,122118,60890,15016,43755,58771,2119,61228
8,충청남도,부여군,68327,46428,9130,35761,44891,1537,21899
9,충청남도,서천군,55805,37802,10536,25308,35844,1958,18003


In [144]:
chungnam_3rd_with_total.to_csv("temp1_governor_chungnam_3.csv", index=False, encoding="utf-8-sig")


## Jeonbuk


In [145]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_jeonbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%A0%84%EB%B6%81.xls'

jeonbuk_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_jeonbuk,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1151

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 라경균
5: 민주당 강현욱
6: 무소속 손주항
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1151행 → 필터링 후: 15행
'투표구명' == '합계'인 행만 추출


In [146]:
jeonbuk_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 라경균,민주당 강현욱,무소속 손주항,_,계,투 표 율 (%),유효 투표 율 (%)
0,전주시완산구,합계,231776,100204,8191,74023,15785,NaN,97999,43.2,97.8
111,전주시덕진구,합계,197458,79884,6453,59409,12470,NaN,78332,40.5,98.1
202,군산시,합계,197267,97041,5055,79646,10182,NaN,94883,49.2,97.8
345,익산시,합계,234964,112465,9121,84169,16164,NaN,109454,47.9,97.3
475,정읍시,합계,105973,66356,4358,48685,11592,NaN,64635,62.6,97.4
571,남원시,합계,75505,52567,3673,36854,10521,NaN,51048,69.6,97.1
664,김제시,합계,87628,58806,8025,40039,8971,NaN,57035,67.1,97
748,완주군,합계,65838,41700,3306,29454,7671,NaN,40431,63.3,97
829,진안군,합계,24876,19517,1905,11812,5194,NaN,18911,78.5,96.9
866,무주군,합계,22378,17272,2063,11219,3507,NaN,16789,77.2,97.2


In [147]:
jeonbuk_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 라경균',
 '민주당 강현욱',
 '무소속 손주항',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [148]:
rename_jeonbuk = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 라경균': '득표수_1_한나라당_라경균',
    '민주당 강현욱': '득표수_2_새천년민주당_강현욱',
    '무소속 손주항': '득표수_3_무소속_손주항',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [149]:
jeonbuk_3rd = jeonbuk_3rd.rename(columns=rename_jeonbuk).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
jeonbuk_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_라경균,득표수_2_새천년민주당_강현욱,득표수_3_무소속_손주항,득표수_계
0,전주시완산구,231776,100204,8191,74023,15785,97999
111,전주시덕진구,197458,79884,6453,59409,12470,78332
202,군산시,197267,97041,5055,79646,10182,94883
345,익산시,234964,112465,9121,84169,16164,109454
475,정읍시,105973,66356,4358,48685,11592,64635
571,남원시,75505,52567,3673,36854,10521,51048
664,김제시,87628,58806,8025,40039,8971,57035
748,완주군,65838,41700,3306,29454,7671,40431
829,진안군,24876,19517,1905,11812,5194,18911
866,무주군,22378,17272,2063,11219,3507,16789


In [150]:
jeonbuk_3rd = jeonbuk_3rd.assign(
    시도='전라북도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + jeonbuk_3rd.columns.tolist()]

In [151]:
jeonbuk_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_라경균,득표수_2_새천년민주당_강현욱,득표수_3_무소속_손주항,득표수_계
0,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999
111,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332
202,전라북도,군산시,197267,97041,5055,79646,10182,94883
345,전라북도,익산시,234964,112465,9121,84169,16164,109454
475,전라북도,정읍시,105973,66356,4358,48685,11592,64635


In [152]:
jeonbuk_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 1092
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                15 non-null     object
 1   구시군               15 non-null     object
 2   선거인수              15 non-null     object
 3   투표수               15 non-null     object
 4   득표수_1_한나라당_라경균    15 non-null     object
 5   득표수_2_새천년민주당_강현욱  15 non-null     object
 6   득표수_3_무소속_손주항     15 non-null     object
 7   득표수_계             15 non-null     object
dtypes: object(8)
memory usage: 1.6+ KB


In [153]:
jeonbuk_3rd = jeonbuk_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [154]:
jeonbuk_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_라경균,득표수_2_새천년민주당_강현욱,득표수_3_무소속_손주항,득표수_계,무효투표수,기권수
0,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999,2205,131572
111,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332,1552,117574
202,전라북도,군산시,197267,97041,5055,79646,10182,94883,2158,100226
345,전라북도,익산시,234964,112465,9121,84169,16164,109454,3011,122499
475,전라북도,정읍시,105973,66356,4358,48685,11592,64635,1721,39617


In [155]:
# 수치형 열만 합계 구하기
summary_row = jeonbuk_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '전라북도')

# summary_row를 맨 위에 붙이기
jeonbuk_3rd_with_total = pd.concat([summary_row, jeonbuk_3rd], ignore_index=True)

In [156]:
jeonbuk_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_라경균,득표수_2_새천년민주당_강현욱,득표수_3_무소속_손주항,득표수_계,무효투표수,기권수
0,전라북도,합계,1431722,787135,63661,571650,131320,766631,20504,644587
1,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999,2205,131572
2,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332,1552,117574
3,전라북도,군산시,197267,97041,5055,79646,10182,94883,2158,100226
4,전라북도,익산시,234964,112465,9121,84169,16164,109454,3011,122499
5,전라북도,정읍시,105973,66356,4358,48685,11592,64635,1721,39617
6,전라북도,남원시,75505,52567,3673,36854,10521,51048,1519,22938
7,전라북도,김제시,87628,58806,8025,40039,8971,57035,1771,28822
8,전라북도,완주군,65838,41700,3306,29454,7671,40431,1269,24138
9,전라북도,진안군,24876,19517,1905,11812,5194,18911,606,5359


In [157]:
jeonbuk_3rd_with_total.to_csv("temp1_governor_jeonbuk_3.csv", index=False, encoding="utf-8-sig")


## Jeonnam


In [158]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_jeonnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%A0%84%EB%82%A8.xls'

jeonnam_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_jeonnam,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 13
데이터 행 수: 1496

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 황수연
5: 민주당 박태영
6: 무소속 송재구
7: 무소속 송하성
8: 무소속 안수원
9: _

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1496행 → 필터링 후: 22행
'투표구명' == '합계'인 행만 추출


In [159]:
jeonnam_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 황수연,민주당 박태영,무소속 송재구,무소속 송하성,무소속 안수원,_,계,투 표 율 (%),유효 투표 율 (%)
0,목포시,합계,169428,81949,2234,48314,24124,4459,1012,NaN,80143,48.4,97.8
108,여수시,합계,224680,128464,5123,73429,32229,11837,2700,NaN,125318,57.2,97.6
275,순천시,합계,185581,102005,4062,55386,22295,13613,3792,NaN,99148,55,97.2
387,나주시,합계,80868,56762,2914,33315,9992,6442,2194,NaN,54857,70.2,96.6
467,광양시,합계,91773,54956,4620,31800,10205,4733,2097,NaN,53455,59.9,97.3
534,담양군,합계,42009,31362,1201,20837,5843,1842,768,NaN,30491,74.7,97.2
579,장성군,합계,41952,30339,1402,22468,3222,1788,717,NaN,29597,72.3,97.6
622,곡성군,합계,30366,24338,1182,14162,5489,1838,855,NaN,23526,80.1,96.7
668,구례군,합계,25437,20009,976,11588,4416,1939,536,NaN,19455,78.7,97.2
708,고흥군,합계,76860,59156,2333,23982,21414,7983,1574,NaN,57286,77,96.8


In [160]:
jeonnam_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 황수연',
 '민주당 박태영',
 '무소속 송재구',
 '무소속 송하성',
 '무소속 안수원',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [161]:
rename_jeonnam = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 황수연': '득표수_1_한나라당_황수연',
    '민주당 박태영': '득표수_2_새천년민주당_박태영',
    '무소속 송재구': '득표수_3_무소속_송재구',
    '무소속 송하성': '득표수_4_무소속_송하성',
    '무소속 안수원': '득표수_5_무소속_안수원',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [162]:
jeonnam_3rd = jeonnam_3rd.rename(columns=rename_jeonnam).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
jeonnam_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_황수연,득표수_2_새천년민주당_박태영,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계
0,목포시,169428,81949,2234,48314,24124,4459,1012,80143
108,여수시,224680,128464,5123,73429,32229,11837,2700,125318
275,순천시,185581,102005,4062,55386,22295,13613,3792,99148
387,나주시,80868,56762,2914,33315,9992,6442,2194,54857
467,광양시,91773,54956,4620,31800,10205,4733,2097,53455
534,담양군,42009,31362,1201,20837,5843,1842,768,30491
579,장성군,41952,30339,1402,22468,3222,1788,717,29597
622,곡성군,30366,24338,1182,14162,5489,1838,855,23526
668,구례군,25437,20009,976,11588,4416,1939,536,19455
708,고흥군,76860,59156,2333,23982,21414,7983,1574,57286


In [163]:
jeonnam_3rd = jeonnam_3rd.assign(
    시도='전라남도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + jeonnam_3rd.columns.tolist()]

In [164]:
jeonnam_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_황수연,득표수_2_새천년민주당_박태영,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계
0,전라남도,목포시,169428,81949,2234,48314,24124,4459,1012,80143
108,전라남도,여수시,224680,128464,5123,73429,32229,11837,2700,125318
275,전라남도,순천시,185581,102005,4062,55386,22295,13613,3792,99148
387,전라남도,나주시,80868,56762,2914,33315,9992,6442,2194,54857
467,전라남도,광양시,91773,54956,4620,31800,10205,4733,2097,53455


In [165]:
jeonnam_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 0 to 1411
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                22 non-null     object
 1   구시군               22 non-null     object
 2   선거인수              22 non-null     object
 3   투표수               22 non-null     object
 4   득표수_1_한나라당_황수연    22 non-null     object
 5   득표수_2_새천년민주당_박태영  22 non-null     object
 6   득표수_3_무소속_송재구     22 non-null     object
 7   득표수_4_무소속_송하성     22 non-null     object
 8   득표수_5_무소속_안수원     22 non-null     object
 9   득표수_계             22 non-null     object
dtypes: object(10)
memory usage: 2.4+ KB


In [166]:
jeonnam_3rd = jeonnam_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [167]:
jeonnam_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_황수연,득표수_2_새천년민주당_박태영,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계,무효투표수,기권수
0,전라남도,목포시,169428,81949,2234,48314,24124,4459,1012,80143,1806,87479
108,전라남도,여수시,224680,128464,5123,73429,32229,11837,2700,125318,3146,96216
275,전라남도,순천시,185581,102005,4062,55386,22295,13613,3792,99148,2857,83576
387,전라남도,나주시,80868,56762,2914,33315,9992,6442,2194,54857,1905,24106
467,전라남도,광양시,91773,54956,4620,31800,10205,4733,2097,53455,1501,36817


In [168]:
# 수치형 열만 합계 구하기
summary_row = jeonnam_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '전라남도')

# summary_row를 맨 위에 붙이기
jeonnam_3rd_with_total = pd.concat([summary_row, jeonnam_3rd], ignore_index=True)

In [169]:
jeonnam_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_황수연,득표수_2_새천년민주당_박태영,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계,무효투표수,기권수
0,전라남도,합계,1533059,1006148,51504,563545,236558,89487,34439,975533,30615,526911
1,전라남도,목포시,169428,81949,2234,48314,24124,4459,1012,80143,1806,87479
2,전라남도,여수시,224680,128464,5123,73429,32229,11837,2700,125318,3146,96216
3,전라남도,순천시,185581,102005,4062,55386,22295,13613,3792,99148,2857,83576
4,전라남도,나주시,80868,56762,2914,33315,9992,6442,2194,54857,1905,24106
5,전라남도,광양시,91773,54956,4620,31800,10205,4733,2097,53455,1501,36817
6,전라남도,담양군,42009,31362,1201,20837,5843,1842,768,30491,871,10647
7,전라남도,장성군,41952,30339,1402,22468,3222,1788,717,29597,742,11613
8,전라남도,곡성군,30366,24338,1182,14162,5489,1838,855,23526,812,6028
9,전라남도,구례군,25437,20009,976,11588,4416,1939,536,19455,554,5428


In [170]:
jeonnam_3rd_with_total.to_csv("temp1_governor_jeonnam_3.csv", index=False, encoding="utf-8-sig")


## Gyeongbuk


In [171]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gyeongbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B2%BD%EB%B6%81.xls'

gyeongbuk_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gyeongbuk,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 1539

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이의근
5: 무소속 조영건
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1539행 → 필터링 후: 24행
'투표구명' == '합계'인 행만 추출


In [172]:
gyeongbuk_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이의근,무소속 조영건,_,계,투 표 율 (%),유효 투표 율 (%)
0,포항시북구,합계,178954,95784,82300,11615,NaN,93915,53.5,98
103,포항시남구,합계,186347,97807,82806,13093,NaN,95899,52.5,98
206,울릉군,합계,7630,6638,5598,917,NaN,6515,87,98.1
219,경주시,합계,210873,125634,107442,15084,NaN,122526,59.6,97.5
336,김천시,합계,110853,71347,59870,9741,NaN,69611,64.4,97.6
418,안동시,합계,136401,91196,74904,13742,NaN,88646,66.9,97.2
524,구미시,합계,231648,100996,83514,15633,NaN,99147,43.6,98.2
653,영주시,합계,95784,65654,55417,8555,NaN,63972,68.5,97.4
732,영천시,합계,90206,52940,41805,9466,NaN,51271,58.7,96.8
807,상주시,합계,93241,65470,53263,10269,NaN,63532,70.2,97


In [173]:
gyeongbuk_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 이의근',
 '무소속 조영건',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [174]:
rename_gyeongbuk = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 이의근': '득표수_1_한나라당_이의근',
    '무소속 조영건': '득표수_3_무소속_조영건',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [175]:
gyeongbuk_3rd = gyeongbuk_3rd.rename(columns=rename_gyeongbuk).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
gyeongbuk_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_이의근,득표수_3_무소속_조영건,득표수_계
0,포항시북구,178954,95784,82300,11615,93915
103,포항시남구,186347,97807,82806,13093,95899
206,울릉군,7630,6638,5598,917,6515
219,경주시,210873,125634,107442,15084,122526
336,김천시,110853,71347,59870,9741,69611
418,안동시,136401,91196,74904,13742,88646
524,구미시,231648,100996,83514,15633,99147
653,영주시,95784,65654,55417,8555,63972
732,영천시,90206,52940,41805,9466,51271
807,상주시,93241,65470,53263,10269,63532


In [176]:
gyeongbuk_3rd = gyeongbuk_3rd.assign(
    시도='경상북도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + gyeongbuk_3rd.columns.tolist()]

In [177]:
gyeongbuk_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이의근,득표수_3_무소속_조영건,득표수_계
0,경상북도,포항시북구,178954,95784,82300,11615,93915
103,경상북도,포항시남구,186347,97807,82806,13093,95899
206,경상북도,울릉군,7630,6638,5598,917,6515
219,경상북도,경주시,210873,125634,107442,15084,122526
336,경상북도,김천시,110853,71347,59870,9741,69611


In [178]:
gyeongbuk_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 1489
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   시도              24 non-null     object
 1   구시군             24 non-null     object
 2   선거인수            24 non-null     object
 3   투표수             24 non-null     object
 4   득표수_1_한나라당_이의근  24 non-null     object
 5   득표수_3_무소속_조영건   24 non-null     object
 6   득표수_계           24 non-null     object
dtypes: object(7)
memory usage: 2.0+ KB


In [179]:
gyeongbuk_3rd = gyeongbuk_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [180]:
gyeongbuk_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이의근,득표수_3_무소속_조영건,득표수_계,무효투표수,기권수
0,경상북도,포항시북구,178954,95784,82300,11615,93915,1869,83170
103,경상북도,포항시남구,186347,97807,82806,13093,95899,1908,88540
206,경상북도,울릉군,7630,6638,5598,917,6515,123,992
219,경상북도,경주시,210873,125634,107442,15084,122526,3108,85239
336,경상북도,김천시,110853,71347,59870,9741,69611,1736,39506


In [181]:
# 수치형 열만 합계 구하기
summary_row = gyeongbuk_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경상북도')

# summary_row를 맨 위에 붙이기
gyeongbuk_3rd_with_total = pd.concat([summary_row, gyeongbuk_3rd], ignore_index=True)

In [182]:
gyeongbuk_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이의근,득표수_3_무소속_조영건,득표수_계,무효투표수,기권수
0,경상북도,합계,2044215,1234095,1028080,174472,1202552,31543,810120
1,경상북도,포항시북구,178954,95784,82300,11615,93915,1869,83170
2,경상북도,포항시남구,186347,97807,82806,13093,95899,1908,88540
3,경상북도,울릉군,7630,6638,5598,917,6515,123,992
4,경상북도,경주시,210873,125634,107442,15084,122526,3108,85239
5,경상북도,김천시,110853,71347,59870,9741,69611,1736,39506
6,경상북도,안동시,136401,91196,74904,13742,88646,2550,45205
7,경상북도,구미시,231648,100996,83514,15633,99147,1849,130652
8,경상북도,영주시,95784,65654,55417,8555,63972,1682,30130
9,경상북도,영천시,90206,52940,41805,9466,51271,1669,37266


In [183]:
gyeongbuk_3rd_with_total.to_csv("temp1_governor_gyeongbuk_3.csv", index=False, encoding="utf-8-sig")


## Gyeongnam


In [184]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gyeongnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B2%BD%EB%82%A8.xls'

gyeongnam_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gyeongnam,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1511

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 김혁규
5: 민주당 김두관
6: 민주노동당 임수태
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1511행 → 필터링 후: 20행
'투표구명' == '합계'인 행만 추출


In [185]:
gyeongnam_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 김혁규,민주당 김두관,민주노동당 임수태,_,계,투 표 율 (%),유효 투표 율 (%)
0,창원시,합계,343825,166981,111414,29630,24139,NaN,165183,48.6,98.9
138,마산시,합계,316267,153345,118410,20247,12621,NaN,151278,48.5,98.7
304,진주시,합계,242187,143459,108529,23928,8679,NaN,141136,59.2,98.4
446,진해시,합계,99953,60998,47057,8519,4555,NaN,60131,61,98.6
502,통영시,합계,98533,56062,44164,7425,3347,NaN,54936,56.9,98
581,고성군(경남),합계,48212,35870,25987,5946,3085,NaN,35018,74.4,97.6
643,사천시,합계,88130,52361,40529,7984,2807,NaN,51320,59.4,98
712,김해시,합계,245332,104406,71692,25517,5753,NaN,102962,42.6,98.6
812,밀양시,합계,92677,56158,43482,7519,3947,NaN,54948,60.6,97.8
898,거제시,합계,122843,64911,44722,11535,7336,NaN,63593,52.8,98


In [186]:
gyeongnam_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 김혁규',
 '민주당 김두관',
 '민주노동당 임수태',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [187]:
rename_gyeongnam = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 김혁규': '득표수_1_한나라당_김혁규',
    '민주당 김두관': '득표수_2_새천년민주당_김두관',
    '민주노동당 임수태': '득표수_3_민주노동당_임수태',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [188]:
gyeongnam_3rd = gyeongnam_3rd.rename(columns=rename_gyeongnam).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
gyeongnam_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_김혁규,득표수_2_새천년민주당_김두관,득표수_3_민주노동당_임수태,득표수_계
0,창원시,343825,166981,111414,29630,24139,165183
138,마산시,316267,153345,118410,20247,12621,151278
304,진주시,242187,143459,108529,23928,8679,141136
446,진해시,99953,60998,47057,8519,4555,60131
502,통영시,98533,56062,44164,7425,3347,54936
581,고성군(경남),48212,35870,25987,5946,3085,35018
643,사천시,88130,52361,40529,7984,2807,51320
712,김해시,245332,104406,71692,25517,5753,102962
812,밀양시,92677,56158,43482,7519,3947,54948
898,거제시,122843,64911,44722,11535,7336,63593


In [189]:
gyeongnam_3rd = gyeongnam_3rd.assign(
    시도='경상남도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + gyeongnam_3rd.columns.tolist()]

In [190]:
gyeongnam_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김혁규,득표수_2_새천년민주당_김두관,득표수_3_민주노동당_임수태,득표수_계
0,경상남도,창원시,343825,166981,111414,29630,24139,165183
138,경상남도,마산시,316267,153345,118410,20247,12621,151278
304,경상남도,진주시,242187,143459,108529,23928,8679,141136
446,경상남도,진해시,99953,60998,47057,8519,4555,60131
502,경상남도,통영시,98533,56062,44164,7425,3347,54936


In [191]:
gyeongnam_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 1459
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                20 non-null     object
 1   구시군               20 non-null     object
 2   선거인수              20 non-null     object
 3   투표수               20 non-null     object
 4   득표수_1_한나라당_김혁규    20 non-null     object
 5   득표수_2_새천년민주당_김두관  20 non-null     object
 6   득표수_3_민주노동당_임수태   20 non-null     object
 7   득표수_계             20 non-null     object
dtypes: object(8)
memory usage: 1.9+ KB


In [192]:
gyeongnam_3rd = gyeongnam_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [193]:
gyeongnam_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김혁규,득표수_2_새천년민주당_김두관,득표수_3_민주노동당_임수태,득표수_계,무효투표수,기권수
0,경상남도,창원시,343825,166981,111414,29630,24139,165183,1798,176844
138,경상남도,마산시,316267,153345,118410,20247,12621,151278,2067,162922
304,경상남도,진주시,242187,143459,108529,23928,8679,141136,2323,98728
446,경상남도,진해시,99953,60998,47057,8519,4555,60131,867,38955
502,경상남도,통영시,98533,56062,44164,7425,3347,54936,1126,42471


In [194]:
# 수치형 열만 합계 구하기
summary_row = gyeongnam_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경상남도')

# summary_row를 맨 위에 붙이기
gyeongnam_3rd_with_total = pd.concat([summary_row, gyeongnam_3rd], ignore_index=True)

In [195]:
gyeongnam_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김혁규,득표수_2_새천년민주당_김두관,득표수_3_민주노동당_임수태,득표수_계,무효투표수,기권수
0,경상남도,합계,2227548,1259142,920706,208641,106367,1235714,23428,968406
1,경상남도,창원시,343825,166981,111414,29630,24139,165183,1798,176844
2,경상남도,마산시,316267,153345,118410,20247,12621,151278,2067,162922
3,경상남도,진주시,242187,143459,108529,23928,8679,141136,2323,98728
4,경상남도,진해시,99953,60998,47057,8519,4555,60131,867,38955
5,경상남도,통영시,98533,56062,44164,7425,3347,54936,1126,42471
6,경상남도,고성군,48212,35870,25987,5946,3085,35018,852,12342
7,경상남도,사천시,88130,52361,40529,7984,2807,51320,1041,35769
8,경상남도,김해시,245332,104406,71692,25517,5753,102962,1444,140926
9,경상남도,밀양시,92677,56158,43482,7519,3947,54948,1210,36519


In [196]:
gyeongnam_3rd_with_total.to_csv("temp1_governor_gyeongnam_3.csv", index=False, encoding="utf-8-sig")


## Jeju

In [197]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_jeju = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%A0%9C%EC%A3%BC.xls'

jeju_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_jeju,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 334

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 신구범
5: 민주당 우근민
6: 민국당 신두완
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 334행 → 필터링 후: 4행
'투표구명' == '합계'인 행만 추출


In [198]:
jeju_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 신구범,민주당 우근민,민국당 신두완,_,계,투 표 율 (%),유효 투표 율 (%)
0,제주시,합계,196356,124337,57387,62036,3402,NaN,122825,63.3,98.8
126,북제주군,합계,74211,55474,24939,27484,1734,NaN,54157,74.8,97.6
208,서귀포시,합계,60910,44602,19471,22759,1690,NaN,43920,73.2,98.5
265,남제주군,합계,56505,43066,17705,23004,1547,NaN,42256,76.2,98.1


In [199]:
jeju_3rd.columns.tolist()

['위원회명',
 '투표구명',
 '선거 인수',
 '투표수',
 '        유효투표수_        후보자별 득표수_한나라당 신구범',
 '민주당 우근민',
 '민국당 신두완',
 '_',
 '계',
 '투 표 율 (%)',
 '유효 투표 율 (%)']

In [200]:
rename_jeju = {
    '위원회명': '구시군',
    '선거 인수': '선거인수',
    '        유효투표수_        후보자별 득표수_한나라당 신구범': '득표수_1_한나라당_신구범',
    '민주당 우근민': '득표수_2_새천년민주당_우근민',
    '민국당 신두완': '득표수_3_민주국민당_신두완',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [201]:
jeju_3rd = jeju_3rd.rename(columns=rename_jeju).drop(columns=['투표구명', '_', '투 표 율 (%)', '유효 투표 율 (%)'])
jeju_3rd

,구시군,선거인수,투표수,득표수_1_한나라당_신구범,득표수_2_새천년민주당_우근민,득표수_3_민주국민당_신두완,득표수_계
0,제주시,196356,124337,57387,62036,3402,122825
126,북제주군,74211,55474,24939,27484,1734,54157
208,서귀포시,60910,44602,19471,22759,1690,43920
265,남제주군,56505,43066,17705,23004,1547,42256


In [202]:
jeju_3rd = jeju_3rd.assign(
    시도='제주도',
    구시군=lambda df: df['구시군'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
)[['시도'] + jeju_3rd.columns.tolist()]

In [203]:
jeju_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_신구범,득표수_2_새천년민주당_우근민,득표수_3_민주국민당_신두완,득표수_계
0,제주도,제주시,196356,124337,57387,62036,3402,122825
126,제주도,북제주군,74211,55474,24939,27484,1734,54157
208,제주도,서귀포시,60910,44602,19471,22759,1690,43920
265,제주도,남제주군,56505,43066,17705,23004,1547,42256


In [204]:
jeju_3rd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to 265
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   시도                4 non-null      object
 1   구시군               4 non-null      object
 2   선거인수              4 non-null      object
 3   투표수               4 non-null      object
 4   득표수_1_한나라당_신구범    4 non-null      object
 5   득표수_2_새천년민주당_우근민  4 non-null      object
 6   득표수_3_민주국민당_신두완   4 non-null      object
 7   득표수_계             4 non-null      object
dtypes: object(8)
memory usage: 460.0+ bytes


In [205]:
jeju_3rd = jeju_3rd.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
    ).assign(
        무효투표수=lambda df: df['투표수'] - df['득표수_계']
    ).assign(
        기권수=lambda df: df['선거인수'] - df['투표수']
    )

In [206]:
jeju_3rd.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_신구범,득표수_2_새천년민주당_우근민,득표수_3_민주국민당_신두완,득표수_계,무효투표수,기권수
0,제주도,제주시,196356,124337,57387,62036,3402,122825,1512,72019
126,제주도,북제주군,74211,55474,24939,27484,1734,54157,1317,18737
208,제주도,서귀포시,60910,44602,19471,22759,1690,43920,682,16308
265,제주도,남제주군,56505,43066,17705,23004,1547,42256,810,13439


In [207]:
# 수치형 열만 합계 구하기
summary_row = jeju_3rd.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '제주도')

# summary_row를 맨 위에 붙이기
jeju_3rd_with_total = pd.concat([summary_row, jeju_3rd], ignore_index=True)

In [208]:
jeju_3rd_with_total

,시도,구시군,선거인수,투표수,득표수_1_한나라당_신구범,득표수_2_새천년민주당_우근민,득표수_3_민주국민당_신두완,득표수_계,무효투표수,기권수
0,제주도,합계,387982,267479,119502,135283,8373,263158,4321,120503
1,제주도,제주시,196356,124337,57387,62036,3402,122825,1512,72019
2,제주도,북제주군,74211,55474,24939,27484,1734,54157,1317,18737
3,제주도,서귀포시,60910,44602,19471,22759,1690,43920,682,16308
4,제주도,남제주군,56505,43066,17705,23004,1547,42256,810,13439


In [209]:
jeju_3rd_with_total.to_csv("temp1_governor_jeju_3.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [210]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp1_governor_incheon_3.csv
Added: temp1_governor_busan_3.csv
Added: temp1_governor_chungnam_3.csv
Added: temp1_governor_jeju_3.csv
Added: temp1_governor_gyeongnam_3.csv
Added: temp1_governor_daegu_3.csv
Added: temp1_governor_jeonbuk_3.csv
Added: temp1_governor_gangwon_3.csv
Added: temp1_governor_daejeon_3.csv
Added: temp1_governor_chungbuk_3.csv
Added: temp1_governor_ulsan_3.csv
Added: temp1_governor_jeonnam_3.csv
Added: temp1_governor_gyeonggi_3.csv
Added: temp1_governor_seoul_3.csv
Added: temp1_governor_gyeongbuk_3.csv
Added: temp1_governor_gwangju_3.csv
Total 16 files compressed.
